Download the dataset here https://www.kaggle.com/arshid/iris-flower-dataset and place it inside `./data/iris`

In [55]:
from torch.utils.data import Dataset, DataLoader
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import pickle
import torch
import pandas as pd
from torch import nn
import torch.nn.functional as F

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [93]:
class IrisDataset():
    def __init__(self):
        df = pd.read_csv("./data/iris/IRIS.csv",skiprows=1)
        labels = df.iloc[:,-1]
        self.x = torch.from_numpy(df.iloc[:,:-1].values.astype("float32"))
        self.y = torch.from_numpy(self._encode_labels(labels)).type(torch.LongTensor)
        self.n_samples = len(df)
        
    def _encode_labels(self,arr):
        encoder = LabelEncoder()
        _encoded = encoder.fit_transform(arr)
        output = open("./data/iris/iris_labels.pkl","wb")
        pickle.dump(encoder,output)
        return _encoded.astype("float32")
        
    def __len__(self):
        return self.n_samples
    
    def __getitem__(self,index):
        return self.x[index], self.y[index]
        

In [94]:
dataset = IrisDataset()
feature,label = dataset[100]
print(feature,label)

tensor([5.8000, 2.7000, 5.1000, 1.9000]) tensor(2)


In [95]:
def train_val_dataset(dataset, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split)
    datasets = {}
    datasets['train'] = Subset(dataset, train_idx)
    datasets['val'] = Subset(dataset, val_idx)
    return datasets

In [96]:
ds = train_val_dataset(dataset)

In [97]:
train_loader = DataLoader(ds['train'].dataset,batch_size=4,shuffle=True)
valid_loader = DataLoader(ds['val'].dataset,batch_size=4*2)

In [144]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(4,4)
    
    def forward(self,xb):
        xb = self.fc1(xb)
        return xb
        
net = Net().to(device)

In [150]:
lossfn = F.cross_entropy
opt = torch.optim.SGD(net.parameters(),1e-2,momentum=0.9)

In [151]:
def fit(n_epochs, model, lossfn, opt, train_dl, valid_dl):
    print("Staring Training")
    for epoch in range(n_epochs):
        running_loss = 0.0
        model.train()
        i = 0
        for xb, yb in train_dl:
            xb = xb.to(device)
            yb = yb.to(device)
            pred = model(xb)
            loss = lossfn(pred,yb)
            opt.zero_grad()
            
            loss.backward()
            # Gradient descent updating weights
            opt.step()
            i+=1
        model.eval()
        
        # print statistics
        running_loss += loss.item()
        # Validation loss
        with torch.no_grad():
            valid_loss = sum(lossfn(model(xb.to(device)), yb.to(device)) for xb, yb in valid_dl)
        print(f"Epoch {epoch+1}/{n_epochs} loss {running_loss} val_loss {valid_loss / len(valid_dl)}")
        running_loss = 0.0
    print("Training Finished.")

In [152]:
fit(50,net,lossfn,opt,train_loader,valid_loader)

Staring Training
Epoch 1/50 loss 1.4166115522384644 val_loss 0.5103714466094971
Epoch 2/50 loss 0.3130640685558319 val_loss 0.39526644349098206
Epoch 3/50 loss 0.378709614276886 val_loss 0.3339281976222992
Epoch 4/50 loss 0.29653477668762207 val_loss 0.2905538082122803
Epoch 5/50 loss 0.048768773674964905 val_loss 0.3764844834804535
Epoch 6/50 loss 0.6326228976249695 val_loss 0.5171579122543335
Epoch 7/50 loss 1.580254077911377 val_loss 0.2976049780845642
Epoch 8/50 loss 0.458731472492218 val_loss 0.207060307264328
Epoch 9/50 loss 0.08319688588380814 val_loss 0.19943152368068695
Epoch 10/50 loss 0.05798394978046417 val_loss 0.18840889632701874
Epoch 11/50 loss 1.1027641296386719 val_loss 0.1869998723268509
Epoch 12/50 loss 0.12191270291805267 val_loss 0.18015141785144806
Epoch 13/50 loss 0.9407714605331421 val_loss 0.1925995945930481
Epoch 14/50 loss 0.7263658046722412 val_loss 0.15742425620555878
Epoch 15/50 loss 0.04012030363082886 val_loss 0.15928487479686737
Epoch 16/50 loss 1.0852

In [160]:
correct = 0
total = 0
with torch.no_grad():
    for data in valid_loader:
        features, labels = data[0].to(device), data[1].to(device)
        outputs = net(features)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network in validation data : %d %%' % (
    100 * correct / total))
print("Total validation data samples", total)

Accuracy of the network in validation data : 97 %
Total validation data samples 149
